In [1]:
import pandas as pd
import pickle
from UMLS import UMLS
from tqdm import tqdm
import numpy as np

In [2]:
umls = UMLS()

Loading Raw MRCONSO Lines


15229292it [00:55, 272717.53it/s]                                                                                                      


Creating mappings between concept IDs for easy access.


 55%|██████████████████████████████████████████████▋                                      | 8357481/15229292 [01:54<18:05, 6328.92it/s]

('A9260151', 'C1673160', 'Leifsonia sp. MN 177', '301553', 'NCBI', 'P', 'PF', 'ENG', 'L5741548')


...

('A26761107', 'C4107309', 'Calomera cristipennis', '1603913', 'NCBI', 'P', 'PF', 'ENG', 'L13209948')


 99%|███████████████████████████████████████████████████████████████████████████████████▌| 15147023/15229292 [03:34<00:23, 3530.64it/s]

('A32062011', 'C5337861', 'unclassified Lunulospora', '2649813', 'NCBI', 'P', 'PF', 'ENG', 'L16346694')


100%|███████████████████████████████████████████████████████████████████████████████████| 15229292/15229292 [03:35<00:00, 70665.27it/s]


AttributeError: 'UMLS' object has no attribute 'load_umls'

In [6]:
scuis = []

for aui, scui in umls.aui2scui.items():
    scuis.append(scui.split('|||')[0])

In [7]:
df = pd.DataFrame(scuis)
df['id'] = range(len(df))
df.groupby(0).count().sort_values('id',ascending=False)

,id
0,
,3436449
M0023808,265
81401,209
M0012822,208
M0012818,203
...,...
2310231,1
2310232,1
2310233,1


In [5]:
aui_dataset = pickle.load(open('/data/Bodenreider_UMLS_DL/Interns/Bernal/aui_pairwise_data_splits.100.p','rb'))

In [6]:
for split, tups in aui_dataset.items():
    
    one_way = []
    two_way = []
    
    if split == 'val':
        split = 'dev'
        
    for aui1, aui2, label in tqdm(tups):

        str1 = umls.aui2str[aui1]
        str2 = umls.aui2str[aui2]

        one_way.append((str1 + ' [SEP] ' + str2, label))
        if split == 'train':
            two_way.append((str1 + ' [SEP] ' + str2, label))
            two_way.append((str2 + ' [SEP] ' + str1, label))
        
    one_way_df = pd.DataFrame(one_way, columns=['sents','labels'])
    one_way_df = one_way_df.sample(len(one_way_df), random_state=np.random.RandomState(42))
    
    if split == 'train':
        two_way_df = pd.DataFrame(two_way, columns=['sents','labels'])
        two_way_df = two_way_df.sample(len(two_way_df), random_state=np.random.RandomState(42))
    else:
        two_way_df = one_way_df
    
    one_way_df.to_csv('/data/Bodenreider_UMLS_DL/Interns/Bernal/data/RW-UVA-2020AB-one-way/{}.tsv'.format(split), sep='\t', quoting=3)
    two_way_df.to_csv('/data/Bodenreider_UMLS_DL/Interns/Bernal/data/RW-UVA-2020AB-two-way/{}.tsv'.format(split), sep='\t', quoting=3)

100%|████████████████████████████████████████████████████████████████████████████████████| 5469322/5469322 [00:11<00:00, 466585.80it/s]
